# Bločne šifre

Cilji laboratorijske vaje so sledeči:
- Spoznati kriptografsko-varne generatorje naključnih vrednosti
- Namestiti in se delno spoznati s knjižnico `cryptography`
- Uporabiti bločno šifro AES v števčnem načinu z naključnim IV
- Spoznati, da simetrične šifre ne dajo nikakršnih zagotovil glede celovitosti in izvesti napad
- Uporabiti bločno šifro AES v načinu veriženja tajnopisnih blokov z naključnim IV

## Kriptografsko-varni generatorji naključnih vrednosti

Vsak generator naključnih vrednosti ni primeren za uproabo v kriptografiji. Tisti, ki so, se imenujejo (kriptografsko-)varni generatorji.

Naključne vrednosti bomo v Pythonu ustvarili s pomočjo modula `secrets`. [Povezava na dokumentacijo.](https://docs.python.org/3/library/secrets.html)

In [1]:
import secrets

Npr. pridobimo 16 bajtov za simetrični ključ ali za vrednost NONCE ali IV.

In [2]:
key = secrets.token_bytes(16)
iv = secrets.token_bytes(16)

print(f"Key: {key.hex()}")
print(f"IV: {iv.hex()}")

Key: fe41249b77f8480f774e1daac76361a8
IV: 73e0ead1465f647bf71152cf3729286a


Alternativno lahko uporabimo tudi funkcijo `os.urandom(int)`, ki vrne naključni niz bajtov želene dolžine.

In [3]:
import os

key = os.urandom(16)
iv = os.urandom(16)

print(f"Key: {key.hex()}")
print(f"IV: {iv.hex()}")

Key: 4552a7202d04fb997f31c649d5533255
IV: ed7e4065148a51a760da4665cc95bcf6


Oba načina sta primerna za ustvarjanje kriptografsko-varnih naključnih vrednosti in bi morala delovati v vseh operacijskih sistemih.

## Knijižnica `cryptography`

Pri tem predmetu bomo v Pythonu uproabljali knjižnico kriptografsko knjižnico [`cryptography`; povezava do dokumentacije.](https://cryptography.io/en/latest) Namestimo jo z ukazom preko ukaznega poziva.

```
$ pip install cryptography
```

Na računalnikih v učilnici moramo paket namestiti v uporabniško mapo.

```
$ pip install --user cryptography
```

V grobem je knjižnica razdeljena v dva dela:

- varni (enostavni) del, ki od razvijalca ne terja veliko konfiguracijskih nastavitev in je primeren tudi za kriptografsko manj vešče uporabnike, in
- nevarni (hazmat) del, kjer se pričakuje, da veste kaj počnete.

Prvi ponuja omejen nabor funkcionalnosti, je lažji za uporabo in v njem težje napravimo traparije. Drugi je veliko bolj zmogljiv, a bolj zahteven za uporabo in v kolikor ne veste, kaj počnete, lahko hitro naredite napako. Te so v kriptografiji drage, saj se običajno razkrijejo, po uspešno izvedenem napadu.

Če menite, da bomo uporabljali enostavni del, se motite. Namen predmeta je, da se izučite za pravilno uporabo _nevarnega_. Znanje bo seveda prenosljivo tudi v druge programske jezike in knjižnice, saj je pogojeno z razumevanjem delovanja kriptografskih gradnikov.

## Naloga 1: Šifriranje z AES-CTR

Začnimo z najbolj enostavnim primerom v kodi: šifriranje AES-CTR, kjer je IV naključno izbran. [Povezava do dokumentacije:](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#) pri kriptografiji je __res nujno, da poznamo dokumentacijo in programiramo po 'občutku'.__

Najprej uvozimo zahtevane pakete.

In [4]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

Za šifriranje potrebujemo
- sporočilo
- ključ
- IV
- šifro -- algoritem $E$

In [5]:
# sporočilo -- ne pozabimo ga postrojiti
msg = "Dober dan!".encode("utf8")

# ključ
k = os.urandom(16)

# IV
iv = os.urandom(16)

# algoritem -- to je odvisno od knjižnice: pri knjižnici cryptography
# je treba narediti objekt Cipher in v njem specificirati šifrirni algoritem
# ter njegov način delovanja
cipher = Cipher(
    algorithms.AES(k),
    modes.CTR(iv))
encryptor = cipher.encryptor()

Šifriranje je sedaj _enostavno_:

1. Vmesnik je zastavljen, da lahko šifriramo večkrat, npr. da nalagamo podatke iz več datotek in nimamo vsega v pomnilniku.
2. Ko pokličemo `encryptor.update(pt)`, se šifrira podan čistopis in vrne nastali tajnopis, a ne nujno ves: lahko se zgodi, da zadnji blok čistopisa ni popoln. V tem primeru se preostali del čistopisa še nahaja nešifriran v objektu `encryptor`.
3. Šifriranje zaključimo s klicem funkcije `encryptor.finalize()` -- takrat se ves morebitni čistopis v objektu `encryptor` šifrira in vrne.

Vse rezultate klicev funkcije `update(pt)` ter na koncu tudi klica `finalize()` dodajamo na seznam bajtov, ki predstavlja končni tajnopis. K temu moramo še dodati IV, sicer ne bo mogoče dešifrirati.

In [6]:
# V našem primeru je celoten čistopis v spremenljivki msg zato potrebujemo le en klic
# metode update in zaključek s finalize
ct = encryptor.update(msg) + encryptor.finalize()

print(f"Ključ: {k.hex()}")
print(f"IV: {iv.hex()}")
print(f"Dolžina čistopisa {len(msg)}, čistopis: {msg.hex()}")
print(f"Dolžina tajnopisa {len(ct)}, tajnopis: {ct.hex()}")

Ključ: 592b92434d020a621ca38db29fa8a8b2
IV: dc0db136fd52305eef573cd869f6ce26
Dolžina čistopisa 10, čistopis: 446f6265722064616e21
Dolžina tajnopisa 10, tajnopis: 0983e8ff258b0beb6f25


Dešifriranje je zelo podobno, potrebujemo:

- Tajnopis
- Ključ
- IV, ki je bil uporabljen pri šifriranju
- Šifro -- algoritem $D$

In [7]:
# spremenljivke k, iv, ct (od zgoraj) zaporedoma predstavljajo ključ, IV in tajnopis

# algoritem dešifriranja
decryptor = Cipher(algorithms.AES(k), modes.CTR(iv)).decryptor()
pt = decryptor.update(ct) + decryptor.finalize()
print("Dešifriran tajnopis:", pt.decode("utf8"))

Dešifriran tajnopis: Dober dan!


Implementirajte funkcijo `enc_aes_ctr(k, pt, iv=None)` skladno z navodili, podanimi v komentarju.

In [8]:
def enc_aes_ctr(k, pt, iv=None):
    """Šifrira čistopis `pt` s ključem `k`. 
    
    Če je `iv` podan (tj. ni nastavljen na None), ga uporabi. Če ni, ustvari novega in uporabi tega.
    
    Vrne par (IV, tajnopis)."""
    
    if iv is None:
        iv = secrets.token_bytes(16)
    
    cipher = Cipher(algorithms.AES(k), modes.CTR(iv))
    encryptor = cipher.encryptor()
    ct = encryptor.update(pt) + encryptor.finalize()

    return cipher.mode.nonce, ct

Enotski test.

In [9]:
def test_enc_aes_ctr():   
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    
    msg = "Hello World!".encode("utf8")
    iv, ct = enc_aes_ctr(key, msg, iv)

    assert iv == bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    assert ct == bytes.fromhex("aa9169004da9689efa74f00d")

test_enc_aes_ctr()

Implementirajte funkcijo `dec_aes_ctr(k, pt, iv)` skladno z navodili, podanimi v komentarju.

In [10]:
def dec_aes_ctr(k, ct, iv):
    """Dešifrira tajnopis s ključem `k` in podanim `IV`.
    
    Vrne čistopis."""
    
    decryptor = Cipher(
        algorithms.AES(k),
        modes.CTR(iv)
    ).decryptor()
    
    return decryptor.update(ct) + decryptor.finalize()

In [11]:
def test_dec_aes_ctr():  
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    ct = bytes.fromhex("aa9169004da9689efa74f00d")
    
    pt = dec_aes_ctr(key, ct, iv)

    assert pt == b"Hello World!"

test_dec_aes_ctr()

## Naloga 2: Šifriranje vsebine
Uporabite funkciji iz prejšnje naloge in šifrirajte vsebino datoteke `data/ic-za-narodov-blagor.txt`.

Vsebino nato shranite v datoteko `data/ic-za-narodov-blagor.txt.enc`. 

Ne pozabite: Poleg samega tajnopisa, morate v datoteko shraniti še IV. Dokaj standarden način je, da IV in nastali tajnopis konkatenirate in skupaj shranite v datoteko. (Ločevanje je potem preprosto: prvih 16 bajtov v datoteki je IV, preostali pa so dejanski tajnopis.)

In [14]:
def read_and_encrypt_file(key, filename):
    """Prebere vsebino datoteke `filename`, jo šifrira z AES-CTR in ključem key ter zapiše nazaj disk. 
    
    Ime šifrirane datoteke je enako kot ime nešifrirane, le da se ji še doda še končnica `.enc`,
    npr `data/ic-za-narodov-blagor.txt` se naj šifrira v  `data/ic-za-narodov-blagor.txt.enc`
    
    Funkcija je vrača rezultata."""
    with open(filename, "rb") as h:
        data = h.read()
        
    iv, ct = enc_aes_ctr(key, data)
    
    ct = iv + ct
    
    with open(filename + ".enc", "wb") as h:
        h.write(ct)  

Naslednja funkcija naj prebere datoteko `filename` (npr. `data/ic-za-narodov-blagor.txt.enc`), vsebino razčleni v IV in tajnopis, slednjega dešifrira in ga vrne kot rezultat.

In [15]:
def read_and_decrypt(key, filename):
    """Prebere vsebino datoteke `filename`, jo dešifrira in vrne čistopis."""
    
    with open(filename, "rb") as h:
        data = h.read()
        
    iv, ct = data[:16], data[16:]
    return dec_aes_ctr(key, ct, iv)    

Spodnji test preveri, ali sta implementaciji zgornjih funkcij pravilni: 
- najprej šifrira datoteko `data/ic-za-narodov-blagor.txt` s funkcijo `read_and_encrypt_file`,
- nato s funkcijo `read_and_decrypt` prebere tajnopis in ga dešifrira,
- ter na koncu preveri ali je dešifrirana vsebina pravilna.

In [16]:
def test_enc_dec_file(key):
    # šifriramo in shranimo na disk
    read_and_encrypt_file(key, "data/ic-za-narodov-blagor.txt")
    
    # preberemo in dešifriramo 
    pt = read_and_decrypt(key, "data/ic-za-narodov-blagor.txt.enc")
    
    with open("data/ic-za-narodov-blagor.txt", "rb") as h:
        data = h.read()
        
    assert data == pt

test_enc_dec_file(os.urandom(16))

## Naloga 3: Gnetljivost tajnopisa

Ne pozabite: AES-CTR je na koncu tokovna šifra -- četudi je narejen iz psevdonaključne permutacije: ustvarimo (psevdo)naključno podlogo in jo z operacijo XOR združimo s čistopisom.

Imamo enako nalogo kot prejšnji teden. Za osvežitev spomina.

---

Kot napadalec bomo spremenili tajnopis, tako da bo sprememba vidna v čistopisu. Zgodba je sledeča.

Ana želi poslati zaupno pošto Boru, vas, ki igrate vlogo napadalca Nandija, pa vsebina sporočila _zares_ zanima.

Na srečo vam gre nekaj reči na roko. 

Prvič, Anin računalnik nima internetne povezave, vaš mobilni telefon pa. Zato ji prijazno ponudite, da zanjo postavite mobilno dostopno točko, preko katere se bo lahko povezala v internet in dostopala do poštnega strežnika. Poštni strežnik bo nato sporočilo dostavil Boru. 

Ana in Bor uporabljata poseben protokol za pošto: protokol FMTP -- Funny Mail Transfer Protocol.  Gre za preprost besedilno-osnovan protokol: prva vrstica označuje naslov prejemnika, druga naslov pošiljatelja, tretja zadevo, četrta je vedno prazna in na koncu je sporočilo.

Vse, kar mora Anin poštni odjemalec storiti, da pošlje sporočilo, je, da strežniku FMTP dostavi besedilni niz, podoben naslednjemu.

```txt
prejemnik@enadomena.com
posiljatelj@drugadomena.com
Zadeva sporocila

<Samo sporocilo>
```

Vse morebitne predhodne ali zaključne presledke v vsaki vrstici poštni strežnik ignorira oz. odstrani pred obdelavo. Na primer, e-pošto zgoraj bi lahko napisali kot spodnjo in ne bi bilo nobene razlike.


```txt
                    prejemnik@enadomena.com               
    posiljatelj@drugadomena.com                    
Zadeva sporocila

<Samo sporocilo>
```

Tretjič, Ana vam v dobri veri -- kakšna naivnost! -- pove, da pošilja elektronsko sporočilo Boru in da uporablja protokol FMTP. (Torej poznate strukturo sporočila in vsebino prve vrstice čistopisa.)

__Tokrat Ana uporablja AES v načinu CTR z naključnim IV.__ Mehanizmov za zagotavljanje celovitosti pravtako ni.

Ker nastopate v vlogi **posrednika**, morate sedaj spremeniti sporočilo tako, da strežnik FMTP ne bo poslal pošte Boru, temveč jo bo poslal na naslov, ki ga nadzirate vi, npr. `nandi@obvlada.si`.

---

(Namig: Če ste nalogo rešili prejšnjikrat, boste tokrat imeli toliko dela, kolikor traja kopiranje rešitve.)

In [18]:
# Ana in Bor si delita skrivni ključ dolžine 16 bajtov
ana_bor_psk = os.urandom(16)

# Ana pripravi sporočilo
email = """bor@student.uni-lj.si
ana@student.uni-lj.si
Hej

Na faksu si pozabil kapo.""".strip()

# In ga šifrira z AES-CTR z naključnim IV
iv, ct = enc_aes_ctr(ana_bor_psk, email.encode("utf8"))

# Nato ga pošlje (to simuliramo) na streznik FMTP
print("Ana pošilja šifrirano sporočilo na strežnik")
print("Tajnopis (HEX):", ct.hex())
print("IV (HEX):", iv.hex())

Ana pošilja šifrirano sporočilo na strežnik
Tajnopis (HEX): 5580de627f2631bb7e4365033035d6b605ccc206cd4b4190126ac520d5256f15586148e2ae071e0c4835a1bed5dd666ef4c0569b1e18a19b246c7d99f47d011d5a9b56d92bb29d5000f4
IV (HEX): 429c28b9977ac7765289ae72b327101c


Sedaj je na potezi napadalec Nandi.

Implementirajte funckijo `change_ct(ct, new_email)`, ki na vhodu prejme tajnopis in email naslov. Spremenite tajnopis tako, da ga bo strežnik FMTP še vedno brez dešifriral, a kot prejemnik ne bo naveden Bor, temveč email naslov, ki je podan v argumentu `new_email`. Predpostavite lahko, da bo `new_email` vedno krajši ali enak od naslova `bor@student.uni-lj.si`.

In [19]:
def change_ct(ct, new_email):
    cur = "bor@student.uni-lj.si"
    new_email = new_email.ljust(len(cur))
    
    l = bytearray(ct)
    
    for i, (c, n) in enumerate(zip(cur, new_email)):
        l[i] = l[i] ^ ord(c) ^ ord(n)
        
    return l

S poganjanjem spodnje celice lahko preverite, ali vaša funkcija deluje pravilno. 

In [20]:
def fmtp_receive(key, ct, iv):
    print("Streznik FMTP prejel sporocilo:", ct.hex())
    pt = dec_aes_ctr(key, ct, iv)
    print("Desifrirano sporocilo:")
    print(pt.decode("utf8"))

fmtp_receive(ana_bor_psk, change_ct(ct, "nandi@obvlada.si"), iv)

Streznik FMTP prejel sporocilo: 598ec24665122bbd6d4170492475ccf24986cc55844b4190126ac520d5256f15586148e2ae071e0c4835a1bed5dd666ef4c0569b1e18a19b246c7d99f47d011d5a9b56d92bb29d5000f4
Desifrirano sporocilo:
nandi@obvlada.si     
ana@student.uni-lj.si
Hej

Na faksu si pozabil kapo.


## Naloga 4: Šifriranje s AES-CBC

Sedaj bomo ponovili zgodbo iz naloge 1, le da bomo tokrat uporabili AES v načinu veriženja tajnopisnih blokov z naključnim IV. 

Gre za način, ki je slabši od CTR in se opušča, a ga kot varnostni inženir moramo poznati.

Implementirajte funkcijo enc_aes_cbc(k, pt, iv=None) skladno z navodili, podanimi v komentarju.

Pozor: Način CBC zahteva, da čistopis pred šifriranjem podložite (angl. padding). Po dešifriranju pa je podlogo potrebno odstraniti. V knjižnici `cryptography` morate to storiti sami, a pri tem lahko uporabite modul [symmetric padding.](https://cryptography.io/en/latest/hazmat/primitives/padding/)

Privzeto podlagamo standardom PKCS#7. Sledi primer.

In [35]:
from cryptography.hazmat.primitives import padding

padder = padding.PKCS7(128).padder()
padded_data = padder.update(b"Tole bosta dva bloka podatkov.")
print(f"Po klicu update: {len(padded_data)}: {padded_data}")

padded_data += padder.finalize()
print(f"Po klicu finalize: {len(padded_data)}: {padded_data}")

Po klicu update: 16: b'Tole bosta dva b'
Po klicu finalize: 32: b'Tole bosta dva bloka podatkov.\x02\x02'


Če želimo podlogo odstraniti uporabimo `unpadder`.

In [36]:
unpadder = padding.PKCS7(128).unpadder()
unpadded_data = unpadder.update(padded_data)
print(f"Po klicu update: {len(unpadded_data)}: {unpadded_data}")

unpadded_data += unpadder.finalize()
print(f"Po klicu finalize: {len(unpadded_data)}: {unpadded_data}")

Po klicu update: 16: b'Tole bosta dva b'
Po klicu finalize: 30: b'Tole bosta dva bloka podatkov.'


Morebitne dodatne informacije poiščite v dokumentaciji.

In [38]:
def enc_aes_cbc(k, pt, iv):
    """Šifrira čistopis s ključem.
    
    Če IV ni podan, ustvari naključnega.
    Kot rezultat vrne par (IV, tajnopis)."""
    
    padder = padding.PKCS7(128).padder()
    pt = padder.update(pt) + padder.finalize()
    
    encryptor = Cipher(algorithms.AES(k), modes.CBC(iv)).encryptor()
    ct = encryptor.update(pt) + encryptor.finalize()

    return iv, ct

In [39]:
def test_enc_aes_cbc():   
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    
    msg = "Hello World!".encode("utf8")
    iv, ct = enc_aes_cbc(key, msg, iv)
    
    assert iv == bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    assert ct == bytes.fromhex("1fc519b6a47b3c562d905965e6e2f465")

test_enc_aes_cbc()

In [41]:
def dec_aes_cbc(k, ct, iv):
    """Dešifrira tajnopis s ključem in vrednostjo IV"""

    dec = Cipher(algorithms.AES(k), modes.CBC(iv)).decryptor()
    
    pt = dec.update(ct) + dec.finalize()
    
    padder = padding.PKCS7(128).unpadder()
    pt = padder.update(pt) + padder.finalize()
    
    return pt

In [42]:
def test_dec_aes_cbc():  
    key = bytes.fromhex("6141fb1142cd0611dd95798fd95352bb")
    iv = bytes.fromhex("dedca9742fa8613c6d42eabdea6edba7")
    ct = bytes.fromhex("1fc519b6a47b3c562d905965e6e2f465")
    
    pt = dec_aes_cbc(key, ct, iv)

    assert pt == b"Hello World!"

test_dec_aes_cbc()